<a href="https://colab.research.google.com/github/cothurn/LCU-LoS-Prediction-based-on-MIMIC-III-with-Keras/blob/master/kerasLOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-Processing


In [0]:
from google.colab import drive
drive.mount('/content/gdrive') 


In [0]:
ls "/content/gdrive/My Drive/MIMIC-III/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/"

In [0]:
import gzip
import csv

In [0]:
f = gzip.open("/content/gdrive/My Drive/MIMIC-III/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/CHARTEVENTS.csv.gz", mode='rt')

In [0]:
csvobj = csv.reader(f,delimiter = ',',quotechar="'")

In [0]:
idDict = dict()
for line in csvobj:
    print(line)
    if line[4] in idDict:
        idDict[line[4]] = idDict[line[4]] + 1
    else:
        idDict[line[4]] = 0

In [0]:
d = open("/content/gdrive/My Drive/MIMIC-III/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/itemCount.csv","w")
for i in idDict.items():
  print(str(i[0])+","+str(i[1])+"\n")

  d.write(str(i[0])+","+str(i[1])+"\n")
d.close()

In [0]:
f = gzip.open("/content/gdrive/My Drive/MIMIC-III/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/CHARTEVENTS.csv.gz", mode='rt')
csvobj = csv.reader(f,delimiter = ',',quotechar="'")
ICUID = []
for line in csvobj:
  if line[4] == '211':
    if line[3] not in ICUID:
      if()
      ICUID.append(line[3])
print("part 1 done")


In [0]:
len(ICUID)

In [0]:
f = gzip.open("/content/gdrive/My Drive/MIMIC-III/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/CHARTEVENTS.csv.gz", mode='rt')
csvobj = csv.reader(f,delimiter = ',',quotechar="'")
next(csvobj)
charVueInput = []
for line in csvobj:
  if line[3] in ICUID:
    temp = ",".join([str(elem) for elem in line])+"\n"
    charVueInput.append(temp)
    #print(line[0])
  if(int(line[0])%100000==0):
      print(line[0])
print("part2 done")

In [0]:
careVueChart = open("/content/gdrive/My Drive/MIMIC-III/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/CareVueChartEvent.csv","w")
for line in charVueInput:
  careVueChart.write(line)
careVueChart.close()

In [0]:
careVueChart = open("/content/gdrive/My Drive/MIMIC-III/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/CareVueChartEvent.csv","r")
csvobj = csv.reader(careVueChart,delimiter = ',',quotechar="'")
idDict = dict()
for line in csvobj:
    print(line)
    if line[4] in idDict:
        idDict[line[4]] = idDict[line[4]] + 1
    else:
        idDict[line[4]] = 1

In [0]:
import pandas as pd
import regex as re

In [0]:
vitals = pd.read_table("/content/gdrive/My Drive/MIMIC-III/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/vitals.csv",delimiter=",")

In [0]:
forwardVitals = pd.DataFrame.ffill(vitals)
backwardVitals = pd.DataFrame.bfill(vitals)
forwardBackwardVitals = pd.DataFrame.ffill(backwardVitals)
backwardForwardVitals = pd.DataFrame.bfill(forwardVitals)
del vitals

In [0]:
forwardVitals.columns = ["Seq",'ICUID', 'ChartDate', 'ChartTime', 'HeartRate', 'SysBP',
       'DiasBP', 'MeanBP', 'RespRate', 'TempC', 'SpO2', 'Glucose']

In [0]:
full = re.compile('.{3}00.*')
tfList = list()
for index,row in forwardVitals.iterrows():
  if index%2500000 == 0:
    print(index)
  time = row['ChartTime']
  #print(str(time))
  if full.match(time):
    tfList.append(True)
  else:
    tfList.append(False)


In [0]:
forwardVitals['TF'] = tfList

In [0]:
filteredForwardVitals = forwardVitals[forwardVitals['TF']==True]

In [0]:
del filteredForwardVitals["TF"]

In [0]:
filteredForwardVitals.to_csv(r'/content/gdrive/My Drive/MIMIC-III/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/filteredForwardVitals.csv',index=False,header=True)

In [0]:
backwardVitals.columns = ["Seq",'ICUID', 'ChartDate', 'ChartTime', 'HeartRate', 'SysBP','DiasBP', 'MeanBP', 'RespRate', 'TempC', 'SpO2', 'Glucose']

full = re.compile('.{3}00.*')
tfList = list()
for index,row in backwardVitals.iterrows():
  if index%2500000 == 0:
    print(index)
  time = row['ChartTime']
  #print(str(time))
  if full.match(time):
    tfList.append(True)
  else:
    tfList.append(False)

backwardVitals['TF'] = tfList

filteredbackwardVitals = backwardVitals[backwardVitals['TF']==True]

del filteredbackwardVitals["TF"]

filteredbackwardVitals.to_csv(r'/content/gdrive/My Drive/MIMIC-III/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/filteredbackwardVitals.csv',index=False,header=True)


####

forwardBackwardVitals.columns = ["Seq",'ICUID', 'ChartDate', 'ChartTime', 'HeartRate', 'SysBP','DiasBP', 'MeanBP', 'RespRate', 'TempC', 'SpO2', 'Glucose']

full = re.compile('.{3}00.*')
tfList = list()
for index,row in forwardBackwardVitals.iterrows():
  if index%2500000 == 0:
    print(index)
  time = row['ChartTime']
  #print(str(time))
  if full.match(time):
    tfList.append(True)
  else:
    tfList.append(False)

forwardBackwardVitals['TF'] = tfList

filteredforwardBackwardVitals = forwardBackwardVitals[forwardBackwardVitals['TF']==True]

del filteredforwardBackwardVitals["TF"]

filteredforwardBackwardVitals.to_csv(r'/content/gdrive/My Drive/MIMIC-III/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/filteredforwardBackwardVitals.csv',index=False,header=True)

####



In [0]:
backwardForwardVitals.columns = ["Seq",'ICUID', 'ChartDate', 'ChartTime', 'HeartRate', 'SysBP','DiasBP', 'MeanBP', 'RespRate', 'TempC', 'SpO2', 'Glucose']

full = re.compile('.{3}00.*')
tfList = list()
for index,row in backwardForwardVitals.iterrows():
  if index%2500000 == 0:
    print(index)
  time = row['ChartTime']
  #print(str(time))
  if full.match(time):
    tfList.append(True)
  else:
    tfList.append(False)

backwardForwardVitals['TF'] = tfList

filteredbackwardForwardVitals = backwardForwardVitals[backwardForwardVitals['TF']==True]

del filteredbackwardForwardVitals["TF"]

filteredbackwardForwardVitals.to_csv(r'/content/gdrive/My Drive/MIMIC-III/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/filteredbackwardForwardVitals.csv',index=False,header=True)


In [0]:
fbvitals = pd.read_table("/content/gdrive/My Drive/MIMIC-III/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/filteredforwardBackwardVitals.csv",delimiter=",")

In [0]:
del fbvitals["Seq"]

In [0]:
fbvitals.iloc[0:10]

In [0]:
perPatientList = []
startCount = 1
currentID = 200001
endCount = 1
for i in range(1,fbvitals.shape[0]):
  if(currentID!=fbvitals.iloc[i]["ICUID"]):
    perPatientList.append(fbvitals.iloc[startCount:endCount])
    startCount = i
    endCount = i
    currentID = fbvitals.iloc[i]["ICUID"]
  else:
    endCount = endCount + 1;
  if(i%100000 == 0):
    print(i)


  


In [0]:
tfList = []
for i in perPatientList:
  if(i.shape[0]>23):
    tfList.append(True)
  else:
    tfList.append(False)

In [0]:
perPatientList[8]

In [0]:
from itertools import compress
longPerPatientVitalList = list(compress(perPatientList,tfList))

In [0]:
firstFirst = pd.read_csv("/content/gdrive/My Drive/MIMIC-III/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/FirstFirst.csv",delimiter=",")
firstFirstList = []
for i in range(1,firstFirst.shape[0]):
  firstFirstList.append(firstFirst.iloc[i][0])

In [0]:
ifList = []
for i in longPerPatientVitalList:
    if(i.iloc[0]["ICUID"] not in firstFirstList):
      ifList.append(False)
    else:
      ifList.append(True)



In [0]:
longFirstFirstPerPatientVitalList = list(compress(longPerPatientVitalList,ifList))

In [0]:
longFirstFirstPerPatientVitalList[0][0:24]

In [0]:
longFirstFirstFirstDayPerPatientVitalList = []
for i in longFirstFirstPerPatientVitalList:
  longFirstFirstFirstDayPerPatientVitalList.append(i.iloc[0:24])
final = pd.concat(longFirstFirstFirstDayPerPatientVitalList,axis=0)
final.to_csv(r'/content/gdrive/My Drive/MIMIC-III/Keras/fbFirstDayFirstVisitFirstICU.csv',index=False,header=True)

In [0]:
longFirstFirstFirstDayPerPatientVitalList1 = []
for i in longFirstFirstPerPatientVitalList:
  longFirstFirstFirstDayPerPatientVitalList1.append(i)
final = pd.concat(longFirstFirstFirstDayPerPatientVitalList1,axis=0)
final.to_csv(r'/content/gdrive/My Drive/MIMIC-III/Keras/fbAllDayFirstVisitFirstICU.csv',index=False,header=True)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


In [0]:
#ls "/content/gdrive/My Drive/MIMIC-III/Keras"
import pandas as pd
demo = pd.read_table("/content/gdrive/My Drive/MIMIC-III/Keras/KerasDemographicInput.csv",delimiter = ",")
#matrix = pd.read_table("/content/gdrive/My Drive/MIMIC-III/Keras/fbFirstDayFirstVisitFirstICU.csv",delimiter = ",")
matrix = pd.read_table("/content/gdrive/My Drive/MIMIC-III/Keras/48hMatrixInput.csv",delimiter = ",")
#matrix = pd.read_table("/content/gdrive/My Drive/MIMIC-III/Keras/4hr48hMatrixInput.csv",delimiter = ",")

los = pd.read_table("/content/gdrive/My Drive/MIMIC-III/Keras/KerasDemographicOutput.csv",delimiter = ",")

In [0]:
from sklearn.metrics import roc_curve,roc_auc_score
from __future__ import absolute_import, division, print_function, unicode_literals
from sklearn.metrics import auc
import matplotlib.pyplot as plt

import numpy as np

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

tf.keras.backend.clear_session() 


def plot_roc_curve(fpr,tpr): 
  plt.plot(fpr,tpr) 
  plt.axis([0,1,0,1]) 
  plt.xlabel('False Positive Rate') 
  plt.ylabel('True Positive Rate') 
  plt.show()


In [0]:
del demo['Unnamed: 0']
del los['Unnamed: 0']
del matrix['AbsTime']
del matrix['RelTime']

In [0]:
matrixList = []
startCount = 0
currentID = matrix.iloc[0]["ICUID"]
endCount = 1
for i in range(1,matrix.shape[0]):
  if(currentID!=matrix.iloc[i]["ICUID"]):
    #print(endCount)
    matrixList.append(matrix.iloc[startCount:endCount])
    startCount = i
    endCount = i+1
    currentID = matrix.iloc[i]["ICUID"]
  else:
    endCount = endCount + 1;
  if(i%50000 == 0):
    print(i)

In [0]:
from progressbar import ProgressBar
import math
meanLoS = los['los'].sum()/(los.shape[0])
pbar = ProgressBar()
inputList = []
losIntList = []
for mEle in pbar(matrixList):
  tempList = []
  ICUID = mEle.iloc[0]["ICUID"]
  tempList.append(mEle.drop(columns = ['ICUID','ChartDate','ChartTime'],errors='ignore'))
  tempList.append(demo.loc[demo['ICU.ID'] == ICUID].drop(columns = ['ICU.ID']))#,'Unnamed: 0']))
  tempLos = los.loc[los['ICU.ID'] == ICUID]['los']
  if(tempLos.shape[0]!=1):
    continue
  tempLosList = []
  for i in range(2,15): 
    if math.ceil(tempLos) == 2+i:
      tempLosList.append(True)
    else:
      tempLosList.append(False)
  if math.ceil(tempLos)>15:
    tempLosList.append(True)
  else:
    tempLosList.append(False)
  tempList.append(tempLosList)
  tempList.append(tempLos-1)
 
  if(math.ceil(tempLos)<3):
    tempList.append(True)
  else:
    tempList.append(False)
  if(math.ceil(tempLos)<5):
    tempList.append(True)
  else:
    tempList.append(False)
  if(math.ceil(tempLos)<7):
    tempList.append(True)
  else:
    tempList.append(False)
  losIntList.append(math.ceil(tempLos))
  inputList.append(tempList)
  


In [0]:
import matplotlib.pyplot as plt
import statistics

n, bins, patches = plt.hist(x=outputNumList, bins='auto',
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Days')
plt.ylabel('Frequency')
plt.title('LoS Histogram')
maxfreq = n.max()/2
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)
statistics.mean(outputNumList)

In [0]:
import random
random.seed(2110)
random.shuffle(inputList)
cutoff = int(len(inputList)*2/10)
print(cutoff)
testList = inputList[:cutoff]
trainList = inputList[cutoff:]
copyList = inputList
inputList = trainList

inputDemoList = []
inputMatrixList = []
outputLoSList = []
outputNumList = []
outputBoolList = []
outputBool2List = []
outputBool3List = []
for i in inputList:
  #print(i[0])
  inputDemoList.append(i[1].to_numpy())
  inputMatrixList.append(i[0].to_numpy())
  outputLoSList.append(np.asarray(i[2]))
  outputNumList.append(float(i[3]))
  outputBoolList.append(i[4])
  outputBool2List.append(i[5])
  outputBool3List.append(i[6])



In [0]:
TinputDemoList = []
TinputMatrixList = []
ToutputLoSList = []
ToutputNumList = []
ToutputBoolList = []
ToutputBool2List = []
ToutputBool3List = []
for i in testList:
  TinputDemoList.append(i[1].to_numpy())
  TinputMatrixList.append(i[0].to_numpy())
  ToutputLoSList.append(np.asarray(i[2]))
  ToutputNumList.append(float(i[3]))
  ToutputBoolList.append(i[4])
  ToutputBool2List.append(i[5])
  ToutputBool3List.append(i[6])

In [0]:
inputDemoArray = np.stack(inputDemoList)
inputMatrixArray = np.stack(inputMatrixList)
outputLoSArray = np.stack(outputLoSList)
outputNumArray = np.stack(outputNumList)
outputBoolArray = np.stack(outputBoolList)
outputBool2Array = np.stack(outputBool2List)
outputBool3Array = np.stack(outputBool3List)

In [0]:
TinputDemoArray = np.stack(TinputDemoList)
TinputMatrixArray = np.stack(TinputMatrixList)
ToutputLoSArray = np.stack(ToutputLoSList)
ToutputNumArray = np.stack(ToutputNumList)
ToutputBoolArray = np.stack(ToutputBoolList)
ToutputBool2Array = np.stack(ToutputBool2List)
ToutputBool3Array = np.stack(ToutputBool3List)

In [0]:
np.unique(outputBool2Array,return_counts = True)

In [0]:

#inputDemoArray = inputDemoArray.reshape(inputDemoArray.shape[0],1,75)
#inputMatrixArray = inputMatrixArray.reshape(inputMatrixArray.shape[0],24,8)
outputLoSArray = outputLoSArray.reshape(outputLoSArray.shape[0],1,outputLoSArray.shape[1])
ToutputLoSArray = ToutputLoSArray.reshape(ToutputLoSArray.shape[0],1,ToutputLoSArray.shape[1])


In [0]:
import tensorflow_addons as tfa
demoInput = keras.Input(shape = (1,75),dtype = "float32",name = 'DemoInput')
matrixInput = keras.Input(shape= (24,8),dtype = 'float32',name = 'matrixInput')
convFilter1 = 1024
convKernelSize1 = 2
poolSize = 3
cnnLayer1 = keras.layers.Conv1D(convFilter1,convKernelSize1,input_shape =(24,8))(matrixInput) #23 rows 4 hrs
cnnLayer2 = keras.layers.Conv1D(convFilter1,convKernelSize1)(cnnLayer1) #22 rows 8 hrs
cnnLayer3 = keras.layers.Conv1D(convFilter1,convKernelSize1)(cnnLayer2) #21 rows 16 hrs
cnnLayer4 = keras.layers.Conv1D(convFilter1,(convKernelSize1+1))(cnnLayer3) #19 rows one day
cnnLayer5 = keras.layers.Conv1D(convFilter1,convKernelSize1)(cnnLayer4) # 18 rows
poolingLayer1 = keras.layers.AveragePooling1D(pool_size=poolSize)(cnnLayer5)#6 rows
convFilter3 = 1024
convKernelSize3 = 4
cnnLayer6 = keras.layers.Conv1D(convFilter3,convKernelSize3)(poolingLayer1) #3 rows
poolingLayer2 = keras.layers.AveragePooling1D(pool_size=poolSize)(cnnLayer6) #1 row
#print(keras.Model(inputs = matrixInput,outputs = poolingLayer2).summary())
mergeLayer = keras.layers.concatenate([poolingLayer2,demoInput])
hiddenLayer1 = keras.layers.LocallyConnected1D(2048,1,activation='linear')(mergeLayer)
hiddenLayer2 = keras.layers.LocallyConnected1D(2048,1,activation='linear')(hiddenLayer1)
hiddenLayer3 = keras.layers.Dense(1024,use_bias=True)(hiddenLayer2)
hiddenLayer4 = keras.layers.Dense(1024,use_bias=True)(hiddenLayer3)
finalDenseLayer = keras.layers.Dense(14,activation="softmax", name='Bucket_Output')(hiddenLayer4)
auxDenseLayer = keras.layers.Dense(1,name='Numerical_output')(hiddenLayer4)
boolDenseLayer = keras.layers.Dense(1,name="Boolean_output",activation = 'sigmoid')(hiddenLayer4)
boolDense2Layer = keras.layers.Dense(1,name="Boolean_output2",activation = 'sigmoid')(hiddenLayer4)
boolDense3Layer = keras.layers.Dense(1,name="Boolean_output3",activation = 'sigmoid')(hiddenLayer4)

cnnModel1 = keras.Model(inputs = [demoInput,matrixInput],outputs = [finalDenseLayer,auxDenseLayer])
cnnModel1.compile(optimizer='Adadelta', 
                 loss={'Bucket_Output':'categorical_crossentropy','Numerical_output':'mse'},
                 loss_weights=[1.,.2],
                 metrics={'Bucket_Output':'categorical_accuracy','Numerical_output':keras.metrics.mae})

cnnModel2 = keras.Model(inputs = [demoInput,matrixInput],outputs = finalDenseLayer)
cnnModel2.compile(optimizer='Adadelta', 
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])#,'top_k_categorical_accuracy'])
                 
cnnModel3 = keras.Model(inputs = [demoInput,matrixInput],outputs = boolDenseLayer)
cnnModel3.compile(optimizer='Adadelta',loss='binary_crossentropy',metrics=['accuracy',keras.metrics.AUC(name='auc')])

cnnModel4 = keras.Model(inputs = [demoInput,matrixInput],outputs = boolDense2Layer)
cnnModel4.compile(optimizer='Adadelta',loss='binary_crossentropy',metrics=['accuracy',keras.metrics.AUC(name='auc')])

cnnModel5 = keras.Model(inputs = [demoInput,matrixInput],outputs = boolDense3Layer)
cnnModel5.compile(optimizer='Adadelta',loss='binary_crossentropy',metrics=['accuracy',keras.metrics.AUC(name='auc')])

cnnModel6 = keras.Model(inputs = [demoInput,matrixInput],
                        outputs = [boolDense2Layer,boolDense3Layer])

cnnModel6.compile(optimizer='Adadelta',loss='binary_crossentropy',metrics=['accuracy',keras.metrics.AUC(name='auc')],
                        loss_weights=[1.,1.])



#tf.keras.utils.plot_model(cnnModel6)

In [0]:
tf.keras.backend.clear_session() 

In [0]:
cnnModel1.fit([inputDemoArray,inputMatrixArray],[outputLoSArray,outputNumArray],batch_size= 512, epochs= 64,validation_split = 0.1,shuffle = True)

In [0]:
cnnModel2.fit([inputDemoArray,inputMatrixArray],outputLoSArray,batch_size= 512, epochs= 128,validation_split = 0.2, shuffle = True)

In [0]:
cnnModel4.fit([inputDemoArray,inputMatrixArray],outputBool2Array,batch_size= 256, epochs= 128,validation_split = 0.2, shuffle = True)

In [0]:
cnnModel5.fit([inputDemoArray,inputMatrixArray],outputBool3Array,batch_size= 256, epochs= 128,validation_split = 0.3, shuffle = True)

In [0]:
cnnModel6.fit([inputDemoArray,inputMatrixArray],[outputBool2Array,outputBool3Array],batch_size= 128, epochs= 64,validation_split = 0.2, shuffle = True)

In [0]:
print(cnnModel4.evaluate([TinputDemoArray,TinputMatrixArray],ToutputBool2Array))
y_pred_keras = cnnModel4.predict([TinputDemoArray,TinputMatrixArray]).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(ToutputBool2Array, y_pred_keras)
plot_roc_curve(fpr_keras, tpr_keras)

In [0]:
print(cnnModel5.evaluate([TinputDemoArray,TinputMatrixArray],ToutputBool3Array))
y_pred_keras = cnnModel5.predict([TinputDemoArray,TinputMatrixArray]).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(ToutputBool3Array, y_pred_keras)
plot_roc_curve(fpr_keras, tpr_keras)

In [0]:
cnnModel6.fit([TinputDemoArray,TinputMatrixArray],[ToutputBool2Array,ToutputBool3Array])

In [0]:
print(cnnModel6.fit([TinputDemoArray,TinputMatrixArray],[ToutputBool2Array,ToutputBool3Array]))
(y_pred_keras1,y_pred_keras2) = cnnModel6.predict([TinputDemoArray,TinputMatrixArray])
y_pred_keras1 = y_pred_keras1.ravel()
y_pred_keras2 = y_pred_keras2.ravel()
fpr_keras1, tpr_keras1, thresholds_keras1 = roc_curve(ToutputBool2Array, y_pred_keras1)
fpr_keras2, tpr_keras2, thresholds_keras2 = roc_curve(ToutputBool3Array, y_pred_keras2)
plot_roc_curve(fpr_keras1, tpr_keras1)
plot_roc_curve(fpr_keras2, tpr_keras2)

In [0]:
cnnModel2.fit([TinputDemoArray,TinputMatrixArray],ToutputLoSArray)

# CNN LoS Prediction Model

##Loading data

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


In [0]:
#ls "/content/gdrive/My Drive/MIMIC-III/Keras"
import pandas as pd
demo = pd.read_table("/content/gdrive/My Drive/MIMIC-III/Keras/KerasDemographicInput.csv",delimiter = ",")
matrix = pd.read_table("/content/gdrive/My Drive/MIMIC-III/Keras/2hr48hMatrixInput.csv",delimiter = ",")
los = pd.read_table("/content/gdrive/My Drive/MIMIC-III/Keras/KerasDemographicOutput.csv",delimiter = ",")
del demo['Unnamed: 0']
del los['Unnamed: 0']
del matrix['AbsTime']
del matrix['RelTime']

In [0]:
pip install skll

In [0]:
from sklearn.metrics import roc_curve,roc_auc_score,auc
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from progressbar import ProgressBar
import math
import matplotlib.pyplot as plt
import statistics
import random
import tensorflow_addons as tfa
import sklearn
import skll

tf.keras.backend.clear_session() 

def plot_roc_curve(fpr,tpr): 
  plt.plot(fpr,tpr) 
  plt.axis([0,1,0,1]) 
  plt.xlabel('False Positive Rate') 
  plt.ylabel('True Positive Rate') 
  plt.show()

In [0]:
pbar = ProgressBar()
matrixList = []
startCount = 0
currentID = matrix.iloc[0]["ICUID"]
endCount = 1
for i in pbar(range(1,matrix.shape[0])):
  if(currentID!=matrix.iloc[i]["ICUID"]):
    #print(endCount)
    matrixList.append(matrix.iloc[startCount:endCount])
    startCount = i
    endCount = i+1
    currentID = matrix.iloc[i]["ICUID"]
  else:
    endCount = endCount + 1;

In [0]:
pbar = ProgressBar()
inputList = []
losIntList = []
for mEle in pbar(matrixList):
  tempList = []
  ICUID = mEle.iloc[0]["ICUID"]
  tempList.append(mEle.drop(columns = ['ICUID','ChartDate','ChartTime'],errors='ignore'))
  tempList.append(demo.loc[demo['ICU.ID'] == ICUID].drop(columns = ['ICU.ID']))#,'Unnamed: 0']))
  tempLos = los.loc[los['ICU.ID'] == ICUID]['los']
  if(tempLos.shape[0]!=1):
    continue
  tempLosList = []
  for i in range(2,15): 
    if math.ceil(tempLos) == 2+i:
      tempLosList.append(True)
    else:
      tempLosList.append(False)
  if math.ceil(tempLos)>15:
    tempLosList.append(True)
  else:
    tempLosList.append(False)
  tempList.append(tempLosList)
  tempList.append(tempLos-1)
 
  if(math.ceil(tempLos)<4):
    tempList.append(True)
  else:
    tempList.append(False)
  if(math.ceil(tempLos)<5):
    tempList.append(True)
  else:
    tempList.append(False)
  if(math.ceil(tempLos)<8):
    tempList.append(True)
  else:
    tempList.append(False)
  if(math.ceil(tempLos)<10):
    tempList.append(True)
  else:
    tempList.append(False)
  losIntList.append(math.ceil(tempLos))
  inputList.append(tempList)
  


## mean and median

In [0]:
inputDemoList = []
inputMatrixList = []
outputLoSList = []
outputNumList = []
outputBoolList = []
outputBool2List = []
outputBool3List = []
outputBool4List = []
for i in inputList:
  #print(i[0])
  inputDemoList.append(i[1].to_numpy())
  inputMatrixList.append(i[0].to_numpy())
  outputLoSList.append(np.asarray(i[2]))
  outputNumList.append(float(i[3]))
  outputBoolList.append(i[4])
  outputBool2List.append(i[5])
  outputBool3List.append(i[6])
  outputBool4List.append(i[7])

In [0]:
n, bins, patches = plt.hist(x=outputNumList, bins='auto',
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Days')
plt.ylabel('Frequency')
plt.title('LoS Histogram')
maxfreq = n.max()/2
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)
print(statistics.mean(outputNumList))
print(statistics.median(outputNumList))

In [0]:
print(statistics.mean(outputBoolList))
print(statistics.pstdev(outputBoolList))
print(statistics.mean(outputBool2List))
print(statistics.pstdev(outputBool2List))
print(statistics.mean(outputBool3List))
print(statistics.pstdev(outputBool3List))


In [0]:
inputDemoArray = np.concatenate(inputDemoList)

In [0]:

counter = collections.Counter(inputDemoArray[:,3])
print(counter)

In [0]:
valList = inputDemoArray[:,59].tolist()
print(statistics.mean(valList))
print(statistics.pstdev(valList))

In [0]:
import collections
heartRate = inputMatrixArray[:,7]
heartRate = heartRate.tolist()
newlist = []
for i in heartRate:
  if i < 5000:
    newlist.append(i)
n, bins, patches = plt.hist(newlist, 100, facecolor='blue')
plt.show()
print(statistics.mean(newlist))
print(statistics.pstdev(newlist))

In [0]:

heartRate = [int(x) for x in heartRate]
counter = collections.Counter(heartRate)
print(counter)
n, bins, patches = plt.hist(heartRate[0:100000], 100, facecolor='blue')
plt.show()

## split

In [0]:
random.seed(2110)
random.shuffle(inputList)
cutoff = int(len(inputList)*2/10)
print(cutoff)
testList = inputList[:cutoff]
trainList = inputList[cutoff:]
copyList = inputList
inputList = trainList

inputDemoList = []
inputMatrixList = []
outputLoSList = []
outputNumList = []
outputBoolList = []
outputBool2List = []
outputBool3List = []
outputBool4List = []
for i in inputList:
  #print(i[0])
  inputDemoList.append(i[1].to_numpy())
  inputMatrixList.append(i[0].to_numpy())
  outputLoSList.append(np.asarray(i[2]))
  outputNumList.append(float(i[3]))
  outputBoolList.append(i[4])
  outputBool2List.append(i[5])
  outputBool3List.append(i[6])
  outputBool4List.append(i[7])

TinputDemoList = []
TinputMatrixList = []
ToutputLoSList = []
ToutputNumList = []
ToutputBoolList = []
ToutputBool2List = []
ToutputBool3List = []
ToutputBool4List = []
for i in testList:
  TinputDemoList.append(i[1].to_numpy())
  TinputMatrixList.append(i[0].to_numpy())
  ToutputLoSList.append(np.asarray(i[2]))
  ToutputNumList.append(float(i[3]))
  ToutputBoolList.append(i[4])
  ToutputBool2List.append(i[5])
  ToutputBool3List.append(i[6])
  ToutputBool4List.append(i[7])

In [0]:
inputDemoArray = np.stack(inputDemoList)
inputMatrixArray = np.stack(inputMatrixList)
outputLoSArray = np.stack(outputLoSList)
outputNumArray = np.stack(outputNumList)
outputBoolArray = np.stack(outputBoolList)
outputBool2Array = np.stack(outputBool2List)
outputBool3Array = np.stack(outputBool3List)
outputBool4Array = np.stack(outputBool4List)

In [0]:
TinputDemoArray = np.stack(TinputDemoList)
TinputMatrixArray = np.stack(TinputMatrixList)
ToutputLoSArray = np.stack(ToutputLoSList)
ToutputNumArray = np.stack(ToutputNumList)
ToutputBoolArray = np.stack(ToutputBoolList)
ToutputBool2Array = np.stack(ToutputBool2List)
ToutputBool3Array = np.stack(ToutputBool3List)
ToutputBool4Array = np.stack(ToutputBool4List)

In [0]:
np.unique(outputBoolArray,return_counts = True)

In [0]:
np.unique(outputBool2Array,return_counts = True)

In [0]:
np.unique(outputBool3Array,return_counts = True)

In [0]:
np.unique(outputBool4Array,return_counts = True)

In [0]:
outputLoSArray = outputLoSArray.reshape(outputLoSArray.shape[0],1,outputLoSArray.shape[1])
ToutputLoSArray = ToutputLoSArray.reshape(ToutputLoSArray.shape[0],1,ToutputLoSArray.shape[1])


##CNN

In [0]:
demoInput = keras.Input(shape = (1,75),dtype = "float32",name = 'Admin_Data_Input')
matrixInput = keras.Input(shape= (24,8),dtype = 'float32',name = 'Vital_Signs_Input')
convFilter1 = 1024
convKernelSize1 = 2
poolSize = 3
cnnLayer1 = keras.layers.Conv1D(convFilter1,convKernelSize1,input_shape =(24,8),name = 'Conv_Layer_1')(matrixInput) #23 rows 4 hrs
cnnLayer2 = keras.layers.Conv1D(convFilter1,convKernelSize1,name = 'Conv_Layer_2')(cnnLayer1) #22 rows 8 hrs
cnnLayer4 = keras.layers.Conv1D(convFilter1,(convKernelSize1+1),name = 'Conv_Layer_3')(cnnLayer2) #18 rows one day
cnnLayer5 = keras.layers.Conv1D(convFilter1,convKernelSize1,name = 'Conv_Layer_4')(cnnLayer4) # 18 rows
poolingLayer1 = keras.layers.MaxPooling1D(pool_size=poolSize,name = 'Pooling_Layer_1')(cnnLayer5)#6 rows
convFilter3 = 1024
convKernelSize3 = 4
cnnLayer6 = keras.layers.Conv1D(convFilter3,convKernelSize3,name = 'Conv_Layer_5')(poolingLayer1) #3 rows
poolingLayer2 = keras.layers.MaxPooling1D(pool_size=poolSize,name = 'Pooling_Layer_2')(cnnLayer6) #1 row
mergeLayer = keras.layers.concatenate([poolingLayer2,demoInput],name = 'Merge_Layer')
hiddenLayer1 = keras.layers.Dense(2048,use_bias=True,activation='linear',name = 'Dense_Layer_1')(mergeLayer)
hiddenLayer2_1 = keras.layers.Dense(2048,use_bias=True,activation='linear',name = 'Dense_Layer_2_for_3_days')(hiddenLayer1)
hiddenLayer3_1 = keras.layers.Dense(2048,use_bias=True,activation='linear',name = 'Dense_Layer_3_for_3_days')(hiddenLayer2_1)
hiddenLayer4_1 = keras.layers.Dense(2048,use_bias=True,activation='linear',name = 'Dense_Layer_4_for_3_days')(hiddenLayer3_1)

hiddenLayer2_2 = keras.layers.Dense(2048,use_bias=True,activation='linear',name = 'Dense_Layer_2_for_4_days')(hiddenLayer1)
hiddenLayer3_2 = keras.layers.Dense(2048,use_bias=True,activation='linear',name = 'Dense_Layer_3_for_4_days')(hiddenLayer2_2)
hiddenLayer4_2 = keras.layers.Dense(2048,use_bias=True,activation='linear',name = 'Dense_Layer_4_for_4_days')(hiddenLayer3_2)

hiddenLayer2_3 = keras.layers.Dense(2048,use_bias=True,activation='linear',name = 'Dense_Layer_2_for_7_days')(hiddenLayer1)
hiddenLayer3_3 = keras.layers.Dense(2048,use_bias=True,activation='linear',name = 'Dense_Layer_3_for_7_days')(hiddenLayer2_3)
hiddenLayer4_3 = keras.layers.Dense(2048,use_bias=True,activation='linear',name = 'Dense_Layer_4_for_7_days')(hiddenLayer3_3)

hiddenLayer2_4 = keras.layers.Dense(2048,use_bias=True,activation='linear',name = 'Dense_Layer_2_for_9_days')(hiddenLayer1)
hiddenLayer3_4 = keras.layers.Dense(2048,use_bias=True,activation='linear',name = 'Dense_Layer_3_for_9_days')(hiddenLayer2_4)
hiddenLayer4_4 = keras.layers.Dense(2048,use_bias=True,activation='linear',name = 'Dense_Layer_4_for_9_days')(hiddenLayer3_4)


finalDenseLayer = keras.layers.Dense(14,activation="softmax", name='Bucket_Output')(hiddenLayer4_1)
auxDenseLayer = keras.layers.Dense(1,name='Numerical_output')(hiddenLayer4_1)
boolDense1Layer = keras.layers.Dense(1,activation = 'sigmoid',name = 'output_for_3_days')(hiddenLayer4_1)
boolDense2Layer = keras.layers.Dense(1,activation = 'sigmoid',name = 'output_for_4_days')(hiddenLayer4_2)
boolDense3Layer = keras.layers.Dense(1,activation = 'sigmoid',name = 'output_for_7_days')(hiddenLayer4_3)
boolDense4Layer = keras.layers.Dense(1,activation = 'sigmoid',name = 'output_for_9_days')(hiddenLayer4_4)

cnnModel2 = keras.Model(inputs = [demoInput,matrixInput],outputs = finalDenseLayer)
cnnModel2.compile(optimizer='Adadelta', 
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

cnnModel3 = keras.Model(inputs = [demoInput,matrixInput],outputs = boolDense1Layer)
cnnModel3.compile(optimizer='Adadelta',loss='binary_crossentropy',metrics=['accuracy',keras.metrics.AUC(name='auc')])

cnnModel4 = keras.Model(inputs = [demoInput,matrixInput],outputs = boolDense2Layer)
cnnModel4.compile(optimizer='Adadelta',loss='binary_crossentropy',metrics=['accuracy',keras.metrics.AUC(name='auc')])

cnnModel5 = keras.Model(inputs = [demoInput,matrixInput],outputs = boolDense3Layer)
cnnModel5.compile(optimizer='Adadelta',loss='binary_crossentropy',metrics=['accuracy',keras.metrics.AUC(name='auc')])

cnnModel6 = keras.Model(inputs = [demoInput,matrixInput],outputs = boolDense4Layer)
cnnModel6.compile(optimizer='Adadelta',loss='binary_crossentropy',metrics=['accuracy',keras.metrics.AUC(name='auc')])

cnnModel7 = keras.Model(inputs = [demoInput,matrixInput],
                        outputs = [boolDense1Layer,boolDense2Layer,boolDense3Layer,boolDense4Layer])

cnnModel7.compile(optimizer='Adadelta',loss='binary_crossentropy',metrics=['accuracy',keras.metrics.AUC(name='auc')],
                        loss_weights=[1.,1.,1.,1.])




tf.keras.utils.plot_model(cnnModel7)

##Train

In [0]:
tf.keras.backend.clear_session() 

In [0]:
cnnModel2.fit([inputDemoArray,inputMatrixArray],outputLoSArray,batch_size= 512, epochs= 128,validation_split = 0.25, shuffle = True)

In [0]:
cnnModel3.fit([inputDemoArray,inputMatrixArray],outputBoolArray,batch_size= 256, epochs= 128,validation_split = 0.25, shuffle = True)

In [0]:
cnnModel4.fit([inputDemoArray,inputMatrixArray],outputBool2Array,batch_size= 256, epochs= 128,validation_split = 0.25, shuffle = True)

In [0]:
cnnModel5.fit([inputDemoArray,inputMatrixArray],outputBool3Array,batch_size= 256, epochs= 128,validation_split = 0.25, shuffle = True)

In [0]:
cnnModel6.fit([inputDemoArray,inputMatrixArray],outputBool4Array,batch_size= 256, epochs= 128,validation_split = 0.25, shuffle = True)

In [0]:
cnnModel7.fit([inputDemoArray,inputMatrixArray],[outputBoolArray,outputBool2Array,outputBool3Array,outputBool4Array],batch_size= 256, epochs= 128,validation_split = 0.25, shuffle = True)

## Validation

In [0]:
y_pred = cnnModel2.predict([TinputDemoArray,TinputMatrixArray])
kll.metrics.kappa(y_pred, ToutputLoSArray)

In [0]:
print(cnnModel3.evaluate([TinputDemoArray,TinputMatrixArray],ToutputBoolArray))
y_pred_keras = cnnModel3.predict([TinputDemoArray,TinputMatrixArray]).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(ToutputBoolArray, y_pred_keras)
plot_roc_curve(fpr_keras, tpr_keras)

In [0]:
print(cnnModel4.evaluate([TinputDemoArray,TinputMatrixArray],ToutputBool2Array))
y_pred_keras = cnnModel4.predict([TinputDemoArray,TinputMatrixArray]).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(ToutputBool2Array, y_pred_keras)
plot_roc_curve(fpr_keras, tpr_keras)

In [0]:
print(cnnModel5.evaluate([TinputDemoArray,TinputMatrixArray],ToutputBool3Array))
y_pred_keras = cnnModel5.predict([TinputDemoArray,TinputMatrixArray]).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(ToutputBool3Array, y_pred_keras)
plot_roc_curve(fpr_keras, tpr_keras)

In [0]:
print(cnnModel6.evaluate([TinputDemoArray,TinputMatrixArray],ToutputBool4Array))
y_pred_keras = cnnModel6.predict([TinputDemoArray,TinputMatrixArray]).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(ToutputBool4Array, y_pred_keras)
plot_roc_curve(fpr_keras, tpr_keras)

In [0]:
print(cnnModel7.evaluate([TinputDemoArray,TinputMatrixArray],[ToutputBoolArray,ToutputBool2Array,ToutputBool3Array,ToutputBool4Array]))
(y_pred_keras,y_pred_keras1,y_pred_keras2,y_pred_keras3) = cnnModel7.predict([TinputDemoArray,TinputMatrixArray])
y_pred_keras = y_pred_keras.ravel()
y_pred_keras1 = y_pred_keras1.ravel()
y_pred_keras2 = y_pred_keras2.ravel()
y_pred_keras3 = y_pred_keras3.ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(ToutputBoolArray, y_pred_keras1)
fpr_keras1, tpr_keras1, thresholds_keras1 = roc_curve(ToutputBool2Array, y_pred_keras1)
fpr_keras2, tpr_keras2, thresholds_keras2 = roc_curve(ToutputBool3Array, y_pred_keras2)
fpr_keras3, tpr_keras3, thresholds_keras3 = roc_curve(ToutputBool4Array, y_pred_keras3)
plot_roc_curve(fpr_keras, tpr_keras)
plot_roc_curve(fpr_keras1, tpr_keras1)
plot_roc_curve(fpr_keras2, tpr_keras2)
plot_roc_curve(fpr_keras3, tpr_keras3)

In [0]:
(y_pred_keras,y_pred_keras1,y_pred_keras2,y_pred_keras3) = cnnModel7.predict([TinputDemoArray,TinputMatrixArray])
ExamplePatientAdmin = TinputDemoList[1]
print("Example patient:")
print("age: " +  str(ExamplePatientAdmin[0][0]))
print("Gender is female: " +  str(ExamplePatientAdmin[0][3]))
print("Insurance is private: " + str(ExamplePatientAdmin[0][8]))
print("Stayed in ICU for: " + str(ToutputNumList[1]) + " days")
print("Predicted value of dischaged in 3 days: " + str(y_pred_keras[1]))
print("Predicted value of dischaged in 4 days: " + str(y_pred_keras1[1]))
print("Predicted value of dischaged in 7 days: " + str(y_pred_keras2[1]))
print("Predicted value of dischaged in 9 days: " + str(y_pred_keras3[1]))
print("results > 0.5 suggests yes, results < 0.5 suggests no")